In [ ]:
# https://chokkan.github.io/mlnote/classification/01binary.html
# 線形二値分類

In [ ]:
# ロジスティック回帰

# シグモイド関数の計算を大きくさせない実装方法
def sigmoid(a):
    if 0 <= a:
        return 1 / (1 + np.exp(-a))
    else:
        return 1. - 1 / (1 + np.exp(a))

In [1]:
# スパムメールの判別問題(DLから展開まで / 以下コメントアウト)
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
# !unzip smsspamcollection.zip
# !head SMSSpamCollection

--2022-07-31 11:34:36--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
archive.ics.uci.edu (archive.ics.uci.edu) をDNSに問いあわせています... 128.195.10.252
archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 203415 (199K) [application/x-httpd-php]
`smsspamcollection.zip' に保存中

smsspamcollection.z 100%[===================>] 198.65K   534KB/s    in 0.4s    

2022-07-31 11:34:37 (534 KB/s) - `smsspamcollection.zip' へ保存完了 [203415/203415]



In [4]:
import collections

def tokenize(s):
    return [t.rstrip('.') for t in s.split(' ')]

def vectorize(tokens):
    return collections.Counter(tokens)

def readiter(fi):
    for line in fi:
        fields = line.strip('\n').split('\t')
        x = vectorize(tokenize(fields[1]))
        y = fields[0]
        yield x, y

with open('SMSSpamCollection') as fi:
    D = [d for d in readiter(fi)]

In [6]:
# 訓練データ(90%)と評価データ(10%)に分割
from sklearn.model_selection import train_test_split

Dtrain, Dtest = train_test_split(D, test_size=0.1, random_state=0)

In [8]:
print(len(Dtrain), len(Dtest))

5016 558


In [9]:
# データ形式の変換
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import DictVectorizer# 特徴をキー、値をバリューとする辞書オブジェクトから特徴ベクトルに変換する

VX = DictVectorizer()
VY = LabelEncoder()

Xtrain = VX.fit_transform([d[0] for d in Dtrain])
Ytrain = VY.fit_transform([d[1] for d in Dtrain])
Xtest = VX.transform([d[0] for d in Dtest])
Ytest = VY.transform([d[1] for d in Dtest])

In [14]:
Dtrain[10]

(Counter({'I': 1,
          'take': 1,
          'it': 2,
          'we': 3,
          "didn't": 1,
          'have': 2,
          'the': 1,
          'phone': 1,
          'callon': 1,
          'Friday': 1,
          'Can': 1,
          'assume': 1,
          "won't": 1,
          'this': 1,
          'year': 1,
          'now?': 1}),
 'ham')

In [18]:
print(Xtrain[10])
print(VX.feature_names_[12653])#疎行列の12653列目の値が3.0なのでその値を入力して単語が何かを調べることができる
print(Ytrain[10])# hamが0 spamが1に分類されていることがわかる

  (0, 1831)	1.0
  (0, 2385)	1.0
  (0, 2769)	1.0
  (0, 5546)	1.0
  (0, 6110)	1.0
  (0, 6923)	1.0
  (0, 8101)	2.0
  (0, 8587)	2.0
  (0, 9821)	1.0
  (0, 10231)	1.0
  (0, 11832)	1.0
  (0, 11957)	1.0
  (0, 12014)	1.0
  (0, 12653)	3.0
  (0, 12862)	1.0
  (0, 13030)	1.0
we
0


In [19]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(loss='log')# デフォルトだと線形分類モデルを確率的勾配降下法で求める。引数にlogを指定するとロジスティック回帰で求めてくれる
model.fit(Xtrain, Ytrain)

SGDClassifier(loss='log')

In [20]:
model.predict(Xtest[0])

array([0])

In [22]:
# ham および spamの予測をする確率を出力
model.predict_proba(Xtest[0])

array([[0.99699625, 0.00300375]])

In [23]:
model.score(Xtest, Ytest)

0.9695340501792115

In [24]:
msg = "Your account has been credited with 500 FREE Text Messages."
model.predict_proba(VX.transform(vectorize(tokenize(msg))))
# 結果スパムに分類された

array([[0.24056691, 0.75943309]])

In [25]:
model.coef_

array([[-0.86749544, -0.26174098, -0.00545712, ...,  0.23200258,
        -0.1092017 , -0.00643666]])

In [26]:
# 特徴を表す単語とその重みのタプルからなるリストを作成し、重みが小さい順に並べたものを変数Fに格納する。
F = sorted(zip(VX.feature_names_, model.coef_[0]), key=lambda x: x[1])

In [27]:
# ham が 0 で spam が 1 なので、以下の単語を含むメッセージはhamだと予測されやすくなる（上位20単語）
F[:20]# 重みの値が負に大きいトップ20

[('And', -1.1894712778240721),
 ('&lt;#&gt;', -1.1808041659924096),
 ('him', -1.1646106420012505),
 ('me', -1.1264978751075003),
 ('I', -1.0354493346822953),
 ('my', -1.0149912565280772),
 ('–', -0.9742690919386593),
 ('u', -0.9149695768194551),
 ('good', -0.8936145625920425),
 ('i', -0.8825908949979039),
 ('', -0.8674954399888316),
 ('Yes', -0.8375927078282269),
 ("I'll", -0.8279633308929651),
 ('ask', -0.8206909306084917),
 ('x', -0.7900361879850132),
 ('So', -0.7898925089000548),
 ('DA', -0.7663645383795855),
 ('&amp;', -0.7650219941147931),
 ('he', -0.7611917905371056),
 ('wan', -0.7478992713585628)]

In [28]:
# 逆で胡散臭いランキング
F[-20:]

[('85233', 1.5293227895278554),
 ('FREE>Ringtone!Reply', 1.5293227895278554),
 ('Text', 1.6029343458446401),
 ('-', 1.6038765062602784),
 ('To', 1.6325888163533444),
 ('Reply', 1.674692896666446),
 ('146tf150p', 1.6991684630604875),
 ('2/2', 1.6991684630604875),
 ('84484', 1.724734364262307),
 ('ringtoneking', 1.724734364262307),
 ('text', 1.7456809939199431),
 ('service', 1.8219409284861507),
 ('STOP', 1.8226130519701098),
 ('won', 1.862421157830023),
 ('&', 1.8900003590153387),
 ('mobile', 1.8977558167068576),
 ('txt', 1.956664409685015),
 ('Txt', 2.0627941236072638),
 ('now!', 2.072258289508794),
 ('Call', 2.3575453584589345)]